## Projet Greedy Algorithm, NAVARRE Victor CACHERA Alexandre

In [ ]:
#Importation des librairies

import numpy as np 

On commence par décrire l'Algorithme Greedy faible pour les n-withd, on réalise une implémentation récursive

In [ ]:
def dist(f,Vn):
    return(min(np.linalg.norm(fi-f) for fi in Vn))


In [ ]:
def weakgreedy (A,lambda,n):
    Vn=[]
    Vn.append(max(np.linalg.norm(v) for v in A))             #On rajoute f0
    Sigma_i=(max(dist(f,Vn) for f in A))
    for i in range(n):
        for f in range(len(A)):
            if dist(f,Vn)>=lambda*Sigma_i:
                Vn.append(f)
                A.remove(f)
                break
        



    return (1)